In [1]:
import random
from imitation.algorithms import preference_comparisons
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env
from imitation.policies.base import FeedForward32Policy, NormalizeFeaturesExtractor
import gymnasium as gym
from stable_baselines3 import PPO
import numpy as np
from typing import Sequence
import torch as th
import numpy as np

from imitation.data.types import (
    AnyPath,
    TrajectoryPair,
    TrajectoryWithRew,
    TrajectoryWithRewPair,
    Transitions,
)

from imitation.data import rollout



/Users/ivanmakarov/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import seals

In [40]:
class CrossEntropyRewardLossCustom(preference_comparisons.RewardLoss):
    """Compute the cross entropy reward loss."""

    def __init__(self) -> None:
        """Create cross entropy reward loss."""
        super().__init__()

    def forward(
        self,
        fragment_pairs: Sequence[TrajectoryPair],
        preferences: np.ndarray,
        preference_model: preference_comparisons.PreferenceModel,
    ) -> preference_comparisons.LossAndMetrics:
        """Computes the loss.

        Args:
            fragment_pairs: Batch consisting of pairs of trajectory fragments.
            preferences: The probability that the first fragment is preferred
                over the second. Typically 0, 1 or 0.5 (tie).
            preference_model: model to predict the preferred fragment from a pair.

        Returns:
            The cross-entropy loss between the probability predicted by the
                reward model and the target probabilities in `preferences`. Metrics
                are accuracy, and gt_reward_loss, if the ground truth reward is
                available.
        """
        rews_pred = th.empty(2 * len(fragment_pairs), dtype=th.float32)
        rews_true = th.empty(2 * len(fragment_pairs), dtype=th.float32)

        cnt1 = 0
        cnt2 = 0
        
        for fragment in fragment_pairs:
            frag1, frag2 = fragment
            trans1 = rollout.flatten_trajectories([frag1])
            trans2 = rollout.flatten_trajectories([frag2])
            rews_pred[cnt1] = preference_model.rewards(trans1).sum()
            cnt1 += 1
            rews_pred[cnt1] = preference_model.rewards(trans2).sum()
            cnt1 += 1

            rews_true[cnt2] = th.tensor(frag1.rews.sum(), dtype=th.float32)
            cnt2 += 1
            rews_true[cnt2] = th.tensor(frag2.rews.sum(), dtype=th.float32)
            cnt2 += 1


        loss = th.nn.MSELoss()(rews_pred, rews_true)
        
        metrics = {}
        metrics["accuracy"] = th.tensor(0)
        metrics = {key: value.detach().cpu() for key, value in metrics.items()}
        return preference_comparisons.LossAndMetrics(
            loss=loss,
            metrics=metrics,
        )

In [44]:
rng = np.random.default_rng(0)

venv = make_vec_env("seals/CartPole-v0", rng=rng)

reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)

fragmenter = preference_comparisons.RandomFragmenter(
    warning_threshold=0,
    rng=rng,
)
gatherer = preference_comparisons.SyntheticGatherer(rng=rng)
preference_model = preference_comparisons.PreferenceModel(reward_net)
reward_trainer = preference_comparisons.BasicRewardTrainer(
    preference_model=preference_model,
    loss=CrossEntropyRewardLossCustom(),
    epochs=3,
    lr=0.001,
    rng=rng,
)


# Several hyperparameters (reward_epochs, ppo_clip_range, ppo_ent_coef,
# ppo_gae_lambda, ppo_n_epochs, discount_factor, use_sde, sde_sample_freq,
# ppo_lr, exploration_frac, num_iterations, initial_comparison_frac,
# initial_epoch_multiplier, query_schedule) used in this example have been
# approximately fine-tuned to reach a reasonable level of performance.
agent = PPO(
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=venv,
    seed=0,
    n_steps=2048 // venv.num_envs,
    batch_size=64,
    ent_coef=0.01,
    learning_rate=2e-3,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    n_epochs=10,
)

trajectory_generator = preference_comparisons.AgentTrainer(
    algorithm=agent,
    reward_fn=reward_net,
    venv=venv,
    exploration_frac=0.05,
    rng=rng,
)

pref_comparisons = preference_comparisons.PreferenceComparisons(
    trajectory_generator,
    reward_net,
    num_iterations=60,  # Set to 60 for better performance
    fragmenter=fragmenter,
    preference_gatherer=gatherer,
    reward_trainer=reward_trainer,
    fragment_length=100,
    transition_oversampling=1,
    initial_comparison_frac=0.1,
    allow_variable_horizon=False,
    initial_epoch_multiplier=4,
    query_schedule="hyperbolic",
)

pref_comparisons.train(
    total_timesteps=50_000,
    total_comparisons=300,
)

Query schedule: [30, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Requested 5700 transitions but only 0 in buffer. Sampling 5700 additional transitions.
Sampling 300 exploratory transitions.
Creating fragment pairs
Gathering preferences
Dataset now contains 30 comparisons


Training reward model:   0%|          | 0/12 [00:00<?, ?it/s]

Training agent for 833 timesteps
---------------------------------------------------
| raw/                                 |          |
|    agent/rollout/ep_rew_wrapped_mean | -296     |
|    agent/time/fps                    | 19750    |
|    agent/time/iterations             | 1        |
|    agent/time/time_elapsed           | 0        |
|    agent/time/total_timesteps        | 2048     |
---------------------------------------------------
---------------------------------------------------
| mean/                                |          |
|    agent/rollout/ep_rew_wrapped_mean | -296     |
|    agent/time/fps                    | 1.98e+04 |
|    agent/time/iterations             | 1        |
|    agent/time/time_elapsed           | 0        |
|    agent/time/total_timesteps        | 2.05e+03 |
|    agent/train/approx_kl             | 0.00201  |
|    agent/train/clip_fraction         | 0.0806   |
|    agent/train/clip_range            | 0.1      |
|    agent/train/entropy_loss  

Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 833 timesteps
-------------------------------------------------------
| raw/                                 |              |
|    agent/rollout/ep_rew_wrapped_mean | -166         |
|    agent/time/fps                    | 22307        |
|    agent/time/iterations             | 1            |
|    agent/time/time_elapsed           | 0            |
|    agent/time/total_timesteps        | 4096         |
|    agent/train/approx_kl             | 0.0020143995 |
|    agent/train/clip_fraction         | 0.0806       |
|    agent/train/clip_range            | 0.1          |
|    agent/train/entropy_loss          | -0.691       |
|    agent/train/explained_variance    | -1.87        |
|    agent/train/learning_rate         | 0.002        |
|    agent/train/loss                  | 0.0105       |
|    agent/train/n_updates             | 10           |
|    agent/train/policy_gradient_loss  | -0.00364     |
|    agent/train/value_loss            | 0.0451       |
---------------

...

Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 853 timesteps
------------------------------------------------------
| raw/                                 |             |
|    agent/rollout/ep_rew_wrapped_mean | 31.4        |
|    agent/time/fps                    | 22207       |
|    agent/time/iterations             | 1           |
|    agent/time/time_elapsed           | 0           |
|    agent/time/total_timesteps        | 122880      |
|    agent/train/approx_kl             | 0.004942756 |
|    agent/train/clip_fraction         | 0.196       |
|    agent/train/clip_range            | 0.1         |
|    agent/train/entropy_loss          | -0.547      |
|    agent/train/explained_variance    | 0.938       |
|    agent/train/learning_rate         | 0.002       |
|    agent/train/loss                  | -0.000154   |
|    agent/train/n_updates             | 590         |
|    agent/train/policy_gradient_loss  | -0.00223    |
|    agent/train/value_loss            | 0.0187      |
--------------------------------

Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 833 timesteps
-------------------------------------------------------
| raw/                                 |              |
|    agent/rollout/ep_rew_wrapped_mean | 33.5         |
|    agent/time/fps                    | 23061        |
|    agent/time/iterations             | 1            |
|    agent/time/time_elapsed           | 0            |
|    agent/time/total_timesteps        | 124928       |
|    agent/train/approx_kl             | 0.0062115993 |
|    agent/train/clip_fraction         | 0.206        |
|    agent/train/clip_range            | 0.1          |
|    agent/train/entropy_loss          | -0.54        |
|    agent/train/explained_variance    | 0.937        |
|    agent/train/learning_rate         | 0.002        |
|    agent/train/loss                  | 0.0104       |
|    agent/train/n_updates             | 600          |
|    agent/train/policy_gradient_loss  | -0.00466     |
|    agent/train/value_loss            | 0.0326       |
---------------

{'reward_loss': 9.597181844711304, 'reward_accuracy': 0.0}

In [47]:
from imitation.rewards.reward_wrapper import RewardVecEnvWrapper

learned_reward_venv = RewardVecEnvWrapper(venv, reward_net.predict_processed)

learner = PPO(
    seed=0,
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=learned_reward_venv,
    batch_size=64,
    ent_coef=0.01,
    n_epochs=10,
    n_steps=2048 // learned_reward_venv.num_envs,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    learning_rate=2e-3,
)
learner.learn(100_000)  # Note: set to 100_000 to train a proficient expert

from stable_baselines3.common.evaluation import evaluate_policy

n_eval_episodes = 10
reward_mean, reward_std = evaluate_policy(learner.policy, venv, n_eval_episodes)
reward_stderr = reward_std / np.sqrt(n_eval_episodes)
print(f"Reward: {reward_mean:.0f} +/- {reward_stderr:.0f}")

Reward: 500 +/- 0
